# **📌 Step-by-Step Tutorial: Building a Voting DApp on Ethereum**
In this tutorial, we will build a **simple blockchain-based voting DApp** that allows users to:  
✅ **Create a poll** with multiple options.  
✅ **Vote** only once for an option.  
✅ **View live vote results**.  
✅ **Automatically close voting after a deadline** (NEW FEATURE).  
✅ **Use MetaMask to connect directly to the blockchain (NO Infura/Alchemy required).**  

---

# **1️⃣ Setting Up the Development Environment**
### **🔹 Install MetaMask**
- **Download MetaMask**: [https://metamask.io/](https://metamask.io/)
- Create a **new wallet** or import an existing one.
- Switch to **Sepolia Testnet** in MetaMask.
- Get free **SepoliaETH** from the faucet:  
  🔗 [https://sepoliafaucet.com/](https://sepoliafaucet.com/)

---

# **2️⃣ Writing the Smart Contract**
### **📌 Open Remix IDE**
1. Go to **[Remix IDE](https://remix.ethereum.org/)**.
2. Create a new file **Voting.sol**.
3. Copy and paste the following Solidity code:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title Voting Smart Contract
 * @dev This contract allows users to create polls, vote, and close polls after a deadline.
 */
contract Voting {
    struct Poll {
        string question; // The question for the poll
        string[] options; // Array to store poll options
        mapping(uint => uint) votes; // Mapping to store vote count for each option
        mapping(address => bool) hasVoted; // Tracks whether an address has voted
        uint deadline; // Voting deadline (timestamp)
        bool exists; // Ensures poll exists
    }

    address public owner; // Contract owner (admin)
    uint public pollCount; // Total number of polls created
    mapping(uint => Poll) public polls; // Mapping of poll ID to Poll struct

    event PollCreated(uint pollId, string question, string[] options, uint deadline);
    event Voted(uint pollId, uint optionIndex);
    event PollClosed(uint pollId);

    constructor() {
        owner = msg.sender; // Set the contract deployer as the owner
        pollCount = 0;
    }

    /**
     * @dev Creates a new poll
     * @param _question The question for the poll
     * @param _options The available options to vote for
     * @param _duration Voting duration in seconds (e.g., 600 for 10 minutes)
     */
    function createPoll(string memory _question, string[] memory _options, uint _duration) public {
        require(msg.sender == owner, "Only owner can create polls");
        require(_options.length > 1, "At least two options required");

        Poll storage newPoll = polls[pollCount];
        newPoll.question = _question;
        newPoll.options = _options;
        newPoll.exists = true;
        newPoll.deadline = block.timestamp + _duration; // Set deadline (current time + duration)

        emit PollCreated(pollCount, _question, _options, newPoll.deadline);
        pollCount++;
    }

    /**
     * @dev Cast a vote for an option in a poll
     * @param _pollId The ID of the poll
     * @param _optionIndex The index of the chosen option
     */
    function vote(uint _pollId, uint _optionIndex) public {
        require(polls[_pollId].exists, "Poll does not exist");
        require(block.timestamp < polls[_pollId].deadline, "Voting period has ended");
        require(!polls[_pollId].hasVoted[msg.sender], "You have already voted");
        require(_optionIndex < polls[_pollId].options.length, "Invalid option");

        polls[_pollId].votes[_optionIndex]++;
        polls[_pollId].hasVoted[msg.sender] = true;

        emit Voted(_pollId, _optionIndex);
    }

    /**
     * @dev Get total votes for a specific poll option
     */
    function getVotes(uint _pollId, uint _optionIndex) public view returns (uint) {
        return polls[_pollId].votes[_optionIndex];
    }

    /**
     * @dev Get poll details (question and options)
     */
    function getPoll(uint _pollId) public view returns (string memory, string[] memory, uint) {
        return (polls[_pollId].question, polls[_pollId].options, polls[_pollId].deadline);
    }
}
```

---

# **3️⃣ Deploying the Smart Contract**
1. **Compile the contract** in Remix (**Solidity 0.8.x**).
2. Go to **Deploy & Run Transactions**.
3. Select **Injected Web3 (MetaMask)** as Environment.
4. Choose **Sepolia Testnet** in MetaMask.
5. Click **Deploy** and confirm the transaction.
6. Copy the **contract address** after deployment.

---

# **4️⃣ Creating the Frontend**
### **📌 Create `index.html`**
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Blockchain Voting DApp</title>
</head>
<body>
    <h2>Blockchain Voting System</h2>

    <div>
        <input type="text" id="question" placeholder="Enter Poll Question">
        <input type="text" id="option1" placeholder="Option 1">
        <input type="text" id="option2" placeholder="Option 2">
        <input type="number" id="duration" placeholder="Duration (seconds)">
        <button onclick="createPoll()">Create Poll</button>
    </div>

    <div>
        <input type="number" id="pollId" placeholder="Enter Poll ID">
        <button onclick="getPoll()">Get Poll</button>
    </div>

    <div id="pollDetails"></div>

    <div>
        <input type="number" id="votePollId" placeholder="Enter Poll ID">
        <input type="number" id="voteOption" placeholder="Enter Option Number">
        <button onclick="vote()">Vote</button>
    </div>

    <div id="transactionLink"></div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/web3/4.0.0/web3.min.js"></script>
    <script src="app.js"></script>
</body>
</html>
```

---

# **5️⃣ Connecting the Frontend to Smart Contract**
### **📌 Create `app.js`**
```javascript
const contractAddress = "YOUR_CONTRACT_ADDRESS_HERE";
const contractABI = [/* COPY ABI FROM REMIX AFTER DEPLOYMENT */];

let web3 = new Web3(window.ethereum);
let contract = new web3.eth.Contract(contractABI, contractAddress);

// Function to create a poll
async function createPoll() {
    const question = document.getElementById("question").value;
    const option1 = document.getElementById("option1").value;
    const option2 = document.getElementById("option2").value;
    const duration = document.getElementById("duration").value;

    const options = [option1, option2];

    const accounts = await web3.eth.getAccounts();
    const receipt = await contract.methods.createPoll(question, options, duration).send({ from: accounts[0] });

    alert("Poll Created!");
}

// Function to vote
async function vote() {
    const pollId = document.getElementById("votePollId").value;
    const optionIndex = document.getElementById("voteOption").value;

    const accounts = await web3.eth.getAccounts();
    await contract.methods.vote(pollId, optionIndex).send({ from: accounts[0] });

    alert("Vote Casted!");
}
```

---

# **✅ Running the DApp**
1. Open **`index.html`** in a browser.
2. Connect MetaMask (**Sepolia Testnet**).
3. **Create a poll, vote, and track live results on Etherscan!** 🚀

---

# **🎯 Features of This DApp**
✅ **Blockchain-based voting system**  
✅ **Uses MetaMask (No Infura/Alchemy needed)**  
✅ **Voting closes automatically after a deadline**  
✅ **Shows live results**  



To make the **Voting DApp mobile-friendly**, we need to ensure it:  
✅ **Works on both desktop and mobile**  
✅ **Supports MetaMask Mobile** (Ethereum transactions via mobile browsers)  
✅ **Uses a responsive design**  

---

# **🔹 How Will This Work on Mobile?**
- **MetaMask Mobile App** has a built-in browser that allows **DApps to interact with Ethereum directly**.
- We will **update the frontend** to work well on **small screens** using **CSS for responsiveness**.
- The **smart contract remains the same**, but the UI will be **touch-friendly**.
- Users will **connect their MetaMask wallets** directly.

---

## **1️⃣ Writing the Mobile-Friendly Smart Contract**
📌 **This remains the same as the desktop version** (Voting.sol).  

➡️ **Use Remix IDE on your phone or laptop** and deploy it to **Sepolia Testnet**.

---

## **2️⃣ Creating the Mobile-Friendly Frontend**
### **📌 Create `index.html`**
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Mobile Blockchain Voting DApp</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin: 20px;
            background-color: #f4f4f4;
        }
        h2 {
            color: #333;
        }
        .container {
            max-width: 400px;
            margin: auto;
            background: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        input, button {
            width: 100%;
            padding: 10px;
            margin: 5px 0;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            background-color: #007bff;
            color: white;
            font-size: 16px;
            cursor: pointer;
        }
        button:hover {
            background-color: #0056b3;
        }
        .hidden {
            display: none;
        }
        #pollDetails, #transactionLink {
            margin-top: 15px;
        }
    </style>
</head>
<body>
    <h2>Blockchain Voting DApp</h2>

    <div class="container">
        <h3>Create Poll</h3>
        <input type="text" id="question" placeholder="Poll Question">
        <input type="text" id="option1" placeholder="Option 1">
        <input type="text" id="option2" placeholder="Option 2">
        <input type="number" id="duration" placeholder="Duration (seconds)">
        <button onclick="createPoll()">Create Poll</button>
    </div>

    <div class="container">
        <h3>Vote on Poll</h3>
        <input type="number" id="pollId" placeholder="Poll ID">
        <button onclick="getPoll()">Get Poll</button>
        <div id="pollDetails"></div>
        <input type="number" id="voteOption" placeholder="Enter Option Number">
        <button onclick="vote()">Vote</button>
    </div>

    <div class="container">
        <h3>Transaction</h3>
        <div id="transactionLink"></div>
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/web3/4.0.0/web3.min.js"></script>
    <script src="app.js"></script>
</body>
</html>
```

---

## **3️⃣ Connecting the Mobile-Friendly UI to Smart Contract**
### **📌 Create `app.js`**
```javascript
// Contract details
const contractAddress = "YOUR_CONTRACT_ADDRESS_HERE"; // Replace with your deployed contract address
const contractABI = [/* COPY ABI FROM REMIX AFTER DEPLOYMENT */];

let web3;
let contract;

// Function to connect to MetaMask on mobile
async function connectMetaMask() {
    if (window.ethereum) {
        try {
            web3 = new Web3(window.ethereum);
            await window.ethereum.request({ method: "eth_requestAccounts" });
            contract = new web3.eth.Contract(contractABI, contractAddress);
            console.log("Connected to contract:", contract);
        } catch (error) {
            alert("MetaMask access denied!");
        }
    } else {
        alert("Please install MetaMask!");
    }
}

// Automatically connect MetaMask when the page loads
window.onload = async () => {
    await connectMetaMask();
};

// Function to create a poll
async function createPoll() {
    const question = document.getElementById("question").value;
    const option1 = document.getElementById("option1").value;
    const option2 = document.getElementById("option2").value;
    const duration = document.getElementById("duration").value;

    const options = [option1, option2];

    const accounts = await web3.eth.getAccounts();
    const receipt = await contract.methods.createPoll(question, options, duration).send({ from: accounts[0] });

    alert("Poll Created!");
    document.getElementById("transactionLink").innerHTML = `
        <a href="https://sepolia.etherscan.io/tx/${receipt.transactionHash}" target="_blank">
            View Transaction on Etherscan
        </a>
    `;
}

// Function to get poll details
async function getPoll() {
    const pollId = document.getElementById("pollId").value;
    const poll = await contract.methods.getPoll(pollId).call();

    const deadline = new Date(poll[2] * 1000).toLocaleString();
    document.getElementById("pollDetails").innerHTML = `
        <h3>${poll[0]}</h3>
        <p>1. ${poll[1][0]}</p>
        <p>2. ${poll[1][1]}</p>
        <p><strong>Voting closes at:</strong> ${deadline}</p>
    `;
}

// Function to vote
async function vote() {
    const pollId = document.getElementById("pollId").value;
    const optionIndex = document.getElementById("voteOption").value;

    const accounts = await web3.eth.getAccounts();
    try {
        const receipt = await contract.methods.vote(pollId, optionIndex).send({ from: accounts[0] });

        alert("Vote Casted!");
        document.getElementById("transactionLink").innerHTML = `
            <a href="https://sepolia.etherscan.io/tx/${receipt.transactionHash}" target="_blank">
                View Transaction on Etherscan
            </a>
        `;
    } catch (error) {
        alert("Voting Failed: " + error.message);
    }
}
```

---

## **4️⃣ How This Works on Mobile**
1️⃣ **Open MetaMask Mobile App** → Tap the **"Browser"** tab.  
2️⃣ **Paste your DApp link (hosted online or via localhost on your PC)**.  
3️⃣ **Connect MetaMask** (Pop-up will appear asking for wallet connection).  
4️⃣ **Create a poll, vote, and track live results on Etherscan!** 🎉  

---

## **✅ Mobile-Friendly Features**
✔ **MetaMask Mobile Compatible**  
✔ **Touch-friendly UI**  
✔ **Auto-adjusting screen size**  
✔ **Works on any mobile browser** (Chrome, Safari, Firefox)  
✔ **No Infura/Alchemy needed** (direct MetaMask interaction)  

---

## **5️⃣ Hosting the DApp Online (Optional)**
You can **host this DApp for free** so that users can access it **without running it locally**.

### **🔹 Option 1: Host on GitHub Pages**
1. Create a **GitHub repository**.
2. Upload **index.html** and **app.js**.
3. Go to **Settings → Pages → Deploy from Main**.

🔗 Your DApp will be live at `https://yourusername.github.io/voting-dapp/`

### **🔹 Option 2: Host on Netlify**
1. Sign up at **[Netlify](https://www.netlify.com/)**.
2. Drag & drop your **HTML & JS files**.
3. **Get a free live URL**.

---

## **🎯 Final Thoughts**
🎉 You now have a **fully mobile-compatible Ethereum Voting DApp**!  
It **works on any phone**, connects directly to **MetaMask**, and can be **hosted online**.  

Let me know if you need **more features like IPFS storage or multiple-choice voting!** 🚀

# **📌 Next Version: Full-Stack Blockchain Voting DApp with Node.js & JavaScript**  
### **Goal**: Create a **full-stack** Voting DApp with:  
✅ **JavaScript (Frontend with React/Vue or Vanilla JS)**  
✅ **Node.js & Express.js (Backend API)**  
✅ **Ethereum Smart Contract (Solidity)**  
✅ **MetaMask for Blockchain Transactions**  
✅ **Works on Desktop & Mobile**  

---

## **🔹 Overview**
Instead of just a static HTML page, we will create a **Node.js web app** that:  
✔ Uses **Express.js** as a backend API  
✔ Connects to an Ethereum smart contract  
✔ Uses **Web3.js** in the frontend  
✔ Serves a **responsive UI**  

### **🔹 Technology Stack**
- **Frontend**: JavaScript (React.js / Vue.js / Vanilla JS)  
- **Backend**: Node.js + Express.js  
- **Blockchain**: Solidity Smart Contract  
- **Storage**: MongoDB (optional, for storing transaction history)  

---

# **1️⃣ Smart Contract (Voting.sol)**
📌 **Same as before** but ensures it's compatible with Web3.js in a full-stack environment.  

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title Voting Smart Contract
 * @dev Allows users to create polls, vote, and close polls after a deadline.
 */
contract Voting {
    struct Poll {
        string question;
        string[] options;
        mapping(uint => uint) votes;
        mapping(address => bool) hasVoted;
        uint deadline;
        bool exists;
    }

    address public owner;
    uint public pollCount;
    mapping(uint => Poll) public polls;

    event PollCreated(uint pollId, string question, string[] options, uint deadline);
    event Voted(uint pollId, uint optionIndex);
    event PollClosed(uint pollId);

    constructor() {
        owner = msg.sender;
        pollCount = 0;
    }

    function createPoll(string memory _question, string[] memory _options, uint _duration) public {
        require(msg.sender == owner, "Only owner can create polls");
        require(_options.length > 1, "At least two options required");

        Poll storage newPoll = polls[pollCount];
        newPoll.question = _question;
        newPoll.options = _options;
        newPoll.exists = true;
        newPoll.deadline = block.timestamp + _duration;

        emit PollCreated(pollCount, _question, _options, newPoll.deadline);
        pollCount++;
    }

    function vote(uint _pollId, uint _optionIndex) public {
        require(polls[_pollId].exists, "Poll does not exist");
        require(block.timestamp < polls[_pollId].deadline, "Voting period has ended");
        require(!polls[_pollId].hasVoted[msg.sender], "Already voted");
        require(_optionIndex < polls[_pollId].options.length, "Invalid option");

        polls[_pollId].votes[_optionIndex]++;
        polls[_pollId].hasVoted[msg.sender] = true;

        emit Voted(_pollId, _optionIndex);
    }

    function getVotes(uint _pollId, uint _optionIndex) public view returns (uint) {
        return polls[_pollId].votes[_optionIndex];
    }

    function getPoll(uint _pollId) public view returns (string memory, string[] memory, uint) {
        return (polls[_pollId].question, polls[_pollId].options, polls[_pollId].deadline);
    }
}
```

---
# **2️⃣ Backend (Node.js + Express.js API)**
### **📌 Setup Node.js Project**
1️⃣ **Initialize a Node.js project**  
```sh
mkdir voting-dapp && cd voting-dapp
npm init -y
```

2️⃣ **Install dependencies**  
```sh
npm install express cors dotenv web3
```

### **📌 Create `server.js`**
```javascript
require("dotenv").config();
const express = require("express");
const cors = require("cors");
const Web3 = require("web3");

// Load environment variables
const PORT = process.env.PORT || 5000;
const CONTRACT_ADDRESS = process.env.CONTRACT_ADDRESS;
const INFURA_URL = process.env.INFURA_URL;

// Initialize Express app
const app = express();
app.use(cors());
app.use(express.json());

// Connect to Ethereum blockchain using Web3.js
const web3 = new Web3(new Web3.providers.HttpProvider(INFURA_URL));
const contractABI = require("./contractABI.json");
const contract = new web3.eth.Contract(contractABI, CONTRACT_ADDRESS);

// Route: Fetch Poll Details
app.get("/poll/:id", async (req, res) => {
    try {
        const pollId = req.params.id;
        const poll = await contract.methods.getPoll(pollId).call();
        res.json({ question: poll[0], options: poll[1], deadline: poll[2] });
    } catch (error) {
        res.status(500).json({ error: error.message });
    }
});

// Route: Fetch Votes for a Poll
app.get("/poll/:id/votes", async (req, res) => {
    try {
        const pollId = req.params.id;
        let votes = [];
        for (let i = 0; i < 2; i++) {
            votes.push(await contract.methods.getVotes(pollId, i).call());
        }
        res.json({ votes });
    } catch (error) {
        res.status(500).json({ error: error.message });
    }
});

// Start the server
app.listen(PORT, () => console.log(`Server running on port ${PORT}`));
```

### **📌 Create `.env` File**
```sh
PORT=5000
CONTRACT_ADDRESS="YOUR_SMART_CONTRACT_ADDRESS"
INFURA_URL="https://sepolia.infura.io/v3/YOUR_INFURA_PROJECT_ID"
```

---
# **3️⃣ Frontend (JavaScript + Web3.js)**
### **📌 Create `frontend/index.html`**
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Voting DApp</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/web3/4.0.0/web3.min.js"></script>
</head>
<body>
    <h2>Blockchain Voting System</h2>
    <div>
        <input type="number" id="pollId" placeholder="Enter Poll ID">
        <button onclick="getPoll()">Get Poll</button>
    </div>
    <div id="pollDetails"></div>

    <script>
        async function getPoll() {
            const pollId = document.getElementById("pollId").value;
            const response = await fetch(`http://localhost:5000/poll/${pollId}`);
            const poll = await response.json();

            document.getElementById("pollDetails").innerHTML = `
                <h3>${poll.question}</h3>
                <p>1. ${poll.options[0]}</p>
                <p>2. ${poll.options[1]}</p>
            `;
        }
    </script>
</body>
</html>
```

---
# **4️⃣ Running the Full-Stack DApp**
1️⃣ **Start Backend**  
```sh
node server.js
```
2️⃣ **Start Frontend** (Simply open `index.html` in a browser)  

---
# **✅ Features of This Full-Stack DApp**
✔ **Uses Express.js backend to communicate with blockchain**  
✔ **Fully responsive web app (works on desktop & mobile)**  
✔ **Uses Web3.js for blockchain transactions**  
✔ **Supports MetaMask & Sepolia Testnet**  
✔ **Can be deployed on a web server**  

---
# **🚀 Next Steps**


## **🚀 Running the Full-Stack Blockchain Voting DApp on Mobile**  
You can run the **Voting DApp** on **mobile devices** by making sure that:  
✔ **The frontend is hosted online** so it can be accessed via a mobile browser.  
✔ **MetaMask Mobile App** is used to sign transactions.  
✔ **The backend is hosted online** so the frontend can communicate with the blockchain.  

---

## **📌 Step 1: Run Backend on a Public Server**
Since **mobile devices cannot directly connect to localhost**, we need to expose our **Node.js server**.

### **Option 1: Use ngrok (Fastest for Local Testing)**
1️⃣ **Install ngrok** (if not already installed)  
   ```sh
   npm install -g ngrok
   ```  
2️⃣ **Run your backend (server.js)**
   ```sh
   node server.js
   ```
3️⃣ **Expose your localhost to the internet**
   ```sh
   ngrok http 5000
   ```
4️⃣ **Copy the `https://your-ngrok-url.io`**  
   - This will be used in the frontend instead of `http://localhost:5000`.

### **Option 2: Deploy the Backend Online**
You can deploy the backend to a **free cloud hosting service**:
- **Render** ([https://render.com/](https://render.com/))  
- **Railway** ([https://railway.app/](https://railway.app/))  
- **Heroku** (requires credit card verification)

---

## **📌 Step 2: Host the Frontend Online**
You **cannot access `index.html` from localhost on mobile**, so we need to host it.

### **Option 1: Use GitHub Pages (Free)**
1️⃣ **Create a GitHub Repository**  
2️⃣ **Upload `index.html` and `app.js`**  
3️⃣ **Go to GitHub Pages settings**  
4️⃣ **Enable GitHub Pages (Deploy from main branch)**  
5️⃣ **Access your live DApp at:**  
   ```
   https://your-username.github.io/voting-dapp/
   ```

### **Option 2: Use Netlify (Easiest)**
1️⃣ **Sign up at** [Netlify](https://www.netlify.com/)  
2️⃣ **Drag & Drop** your frontend files (index.html + app.js)  
3️⃣ **Deploy & get a free URL**  
4️⃣ Open the **Netlify URL** on **your phone**!

---

## **📌 Step 3: Open the DApp on Mobile**
1️⃣ **Install MetaMask Mobile App** (from Google Play or App Store)  
2️⃣ **Open MetaMask and go to the "Browser" tab**  
3️⃣ **Enter your DApp URL** (`https://your-netlify-url.com`)  
4️⃣ **Connect MetaMask to the DApp**  
5️⃣ **Create polls, vote, and track results! 🎉**  

---

## **✅ Summary: How to Run the DApp on Mobile**
| **Step**          | **What to Do** |
|------------------|--------------|
| **1. Run Backend** | Use `ngrok` or deploy to Render/Railway |
| **2. Host Frontend** | Use **GitHub Pages** or **Netlify** |
| **3. Open on Mobile** | Use MetaMask Mobile **Browser** |

---

## **🚀 Now Your Voting DApp Works on Both Desktop & Mobile!**
Let me know if you want **extra features** like real-time polling updates, voting history, or a React/Vue UI! 🚀🔥